# Supervised Classification

Le but de ce TP est d'utiliser deux méthodes différentes sur des données simulées et de voir l'influence de leur paramètres sur leur résultat.

On commence par les SVMs.

## Support Vector Machines

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC     # "Support Vector Classifier"
from matplotlib.colors import ListedColormap

### Cas séparable

Le premier cas que auquel on s'intéresse est celui où les données sont linéairement séparables. C'est à dire, elles sont générées par
$$ Y = f(X) $$
=======
où $f$ est le signe d'une fonction affine.
Commençons par générer et afficher ces données.

##### Exemple 1

In [ ]:
n=150
X=np.random.randn(n,2)
w=[1,-1]
b=0.5
Y=np.sign((np.dot(X,w)+0.5)).ravel()
plt.scatter(X[:, 0], X[:, 1], c=Y, cmap="spring");

Ici les données sont séparables par une droite. Le module SVC de *scikitlearn* permet de trouver la droite qui passe avec la marge maximale séparant les données. On cherche alors à résoudre le problème suivant:
$$ \min \left\{ \left. \frac{1}{2} \|w\|^2 \right| y_i(<w,x_i>+b)\geq 1, \forall 1\leq i \leq n \right\}$$
-----

In [ ]:
clf = SVC(kernel='linear',C=np.Inf)
clf.fit(X, Y)
print clf.coef_
print clf.intercept_
print clf.coef_/np.linalg.norm(clf.coef_)
print w/np.linalg.norm(w)

**Quel est le lien entre** *clf.intercept_* ** et la formulation mathématique du problème?**

*Réponse:*

**Compléter la fonction** *plot_svc* **suivante afin qu'elle trace la droite portée par $\beta$ et celle trouvée par SVC**

In [ ]:
def plot_svc(clf, ax=None):
    if ax is None:
        ax = plt.gca()
    wstar=clf.coef_.ravel()
    x = np.linspace(plt.xlim()[0], plt.xlim()[1], 30)
    ystar = 
    y = 
    ax.plot(ystar,x,c="red")
    ax.plot(y,x,c="green")

plt.scatter(X[:, 0], X[:, 1], c=Y, cmap="spring");
plot_svc(clf,plt.gca())

#### Cas non séparable

On considère maintenant le cas non séparable; certaines données ne sont pas bien étiquetées pour une séparatation linéaire. On autorise alors quelques erreurs dans la recherche de la marge maximale:
$$ \min \left\{ \left. \frac{1}{2} \|w\|^2 + C \sum_{i=1}^n \xi_i \right| y_i(<w,x_i>+b)\geq 1 - \xi_i, \forall 1\leq i \leq n \right\}$$
-----

Cette minimisation est implémentée dans la module SVC; c'est la raison pour laquelle on a posé *C=np.Inf* plus haut.
On intègre un bruit dans la génération de nos données pour des avoir des données non séparables.

##### Exemple 2

In [ ]:
n=150
X=np.random.randn(n,2)
w=[1,-1]
b=0.5
eps=0.7*np.random.randn(n,1)
Y=np.sign((np.dot(X,w)+0.5).reshape(-1,1)+eps).ravel()
plt.scatter(X[:, 0], X[:, 1], c=Y, cmap="spring");

clf = SVC(kernel='linear')
clf.fit(X, Y)
plot_svc(clf,plt.gca());

Il possible de voir les points pour lesquels la contraintes est active; ce sont les **vecteurs support**. On les retrouve à l'aide de l'attribut 

    support_vectors_

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=Y, cmap='spring')
plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1],
            s=50, facecolors='none');

À l'aide de la fonction ``interact`` d'iPython, on peut voir l'effet de la constante *C* sur la méthode.

In [ ]:
from IPython.html.widgets import interact

n=150
X=np.random.randn(n,2)
w=[1,-1]
b=0.5
eps=np.random.randn(n,1)
Y=np.sign((np.dot(X,w)+0.5).reshape(-1,1)+eps).ravel()


def call_plot_svc(C=1):
    plt.scatter(X[:, 0], X[:, 1], c=Y, cmap="spring");
    clf = SVC(kernel='linear',C=C/100.)
    clf.fit(X, Y)
    plot_svc(clf,plt.gca());

    
interact(call_plot_svc, C=[1, 50], kernel='linear');

On peut aussi générer les données de la manière suivante.
On tire une variable gaussienne centrée en $(0,2)$ puis une centrée en $(-2,0)$.

L'objectif est alors de retrouver suivant quelle loi a été tirée la variable aléatoire.

##### Exemple 3

In [ ]:
n=150
X=np.append(np.random.randn(n,2)+[0,2],np.random.randn(n,2)+[-2,0],axis=0)
w=[1,1]
b=0
Y=np.append(np.zeros(n),np.ones(n))
plt.scatter(X[:, 0], X[:, 1], c=Y, cmap="spring");

clf = SVC(kernel='linear')
clf.fit(X, Y)
plot_svc(clf,plt.gca());

Il n'est pas possible de faire mieux que de scinder l'espace en deux pour retrouver l'origine de la variable alétoire.

#### SVM à noyaux

Les SVMs à noyaux permettent de rendre *linéairement* séparables des données qui ne le sont pas, en les plongeant dans un espace de plus grande dimension.

Cette fois, on définit deux variables aléatoires gaussiennes centrées en $(0,0)$ de variance $1$ et $16$.

L'objectif est encore de retrouver la loi de chacune des variables aléatoires.

##### Exemple 4

In [ ]:
n=150
X=np.append(np.random.randn(n,2),4*np.random.randn(n,2),axis=0)
print X.shape
w=[1,1]
b=0
Y=np.append(np.zeros(n),np.ones(n),axis=0)
plt.scatter(X[:, 0], X[:, 1], c=Y, cmap="spring");

In [ ]:
def plot_color_decision(clf, ax=None):
    if ax is None:
        ax = plt.gca()

    h=0.05
    # Create color maps
    cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA'])
    cmap_bold = ListedColormap(['#FF0000', '#00FF00'])

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, m_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    ax.pcolormesh(xx, yy, Z, cmap=cmap_light)

    ax.scatter(X[:, 0], X[:, 1], c=Y, cmap=cmap_bold)
    # Plot also the training points
    ax.xlim(xx.min(), xx.max())
    ax.ylim(yy.min(), yy.max())
    ax.title("SVM classification")
    ax.show()

In [ ]:
clf = SVC(kernel="linear")
clf.fit(X, Y)
plot_color_decision(clf,plt)

Pour contourner ce problème, on va enoyer les données dans un espace dans lequel elles seront linéairement séparable. On peut faire ça dans un espace de dimension infini (par exemple $L_2$), mais on pour comprendre le principe, on va voir comment on peut envoyer les données dans un espace en 3D. Pour ça on ajoute une troisème coordonnée:

In [ ]:
r = np.exp(-(X[:, 0] ** 2 + X[:, 1] ** 2))

In [ ]:
from mpl_toolkits import mplot3d

def plot_3D(elev=30, azim=30):
    ax = plt.subplot(projection='3d')
    ax.scatter3D(X[:, 0], X[:, 1], r, c=Y, s=50, cmap='spring')
    ax.view_init(elev=elev, azim=azim)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('r')

interact(plot_3D, elev=[-90, 90], azip=(-180, 180));

Il est plus raisonnable ici de vouloir séparer les données par un plan linéaire dans ce plongement en 3D!

Le noyau gaussien (issu d'un plongement des données dans un espace de dimension infinie) donne le résultat suivant.

In [ ]:
clf = SVC()
clf.fit(X, Y)
plot_color_decision(clf,plt)

**Utiliser la fonction**

    interact
    
**pour voir l'influence du paramètre** *gamma* **sur le méthode.**

# K plus proches voisins

**Reprenez les 4 exemples de la méthode SVM et implémentez le méthode des k-plus proches voisins à l'aide de la classe **

    neighbors.KNeighborsClassifier   

In [ ]:
from sklearn import neighbors
help(neighbors.KNeighborsClassifier)